In [1]:
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim.nets import resnet_v1

In [2]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg

--2018-03-19 13:53:02--  https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.153.240, 2620:0:860:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.153.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27183 (27K) [image/jpeg]
Saving to: ‘220px-Lynx_lynx_poing.jpg.1’

220px-Lynx_lynx_poi 100%[===================>]  26.55K  --.-KB/s    in 0.01s   

2018-03-19 13:53:02 (2.44 MB/s) - ‘220px-Lynx_lynx_poing.jpg.1’ saved [27183/27183]



In [40]:
im = Image.open("220px-Lynx_lynx_poing.jpg").resize((224,224))
im = np.array(im)
im = np.expand_dims(im, 0)

In [4]:
!wget http://download.tensorflow.org/models/resnet_v1_152_2016_08_28.tar.gz

--2018-03-19 13:53:11--  http://download.tensorflow.org/models/resnet_v1_152_2016_08_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.9.16, 2607:f8b0:4000:812::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.9.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224342140 (214M) [application/x-tar]
Saving to: ‘resnet_v1_152_2016_08_28.tar.gz.1’

resnet_v1_152_2016_ 100%[===================>] 213.95M   106MB/s    in 2.0s    

2018-03-19 13:53:14 (106 MB/s) - ‘resnet_v1_152_2016_08_28.tar.gz.1’ saved [224342140/224342140]



In [5]:
!tar xvf resnet_v1_152_2016_08_28.tar.gz

resnet_v1_152.ckpt


In [3]:
# Placeholders
input_tensor = tf.placeholder(tf.float32, shape=(None,224,224,3), name='input_image')

In [4]:
# Load the model
sess = tf.Session()
arg_scope = resnet_v1.resnet_arg_scope()
with tf.contrib.slim.arg_scope(arg_scope):
    logits, _ = resnet_v1.resnet_v1_152(input_tensor, num_classes=1000, is_training=False)

In [5]:
probabilities = tf.nn.softmax(logits)

In [6]:
checkpoint_file = 'resnet_v1_152.ckpt'
saver = tf.train.Saver()
saver.restore(sess, checkpoint_file)

INFO:tensorflow:Restoring parameters from resnet_v1_152.ckpt


In [41]:
pred, pred_proba = sess.run([logits,probabilities], feed_dict={input_tensor: im})

ValueError: Cannot feed value of shape (224, 224, 3) for Tensor 'input_image:0', which has shape '(?, 224, 224, 3)'

In [10]:
pred_proba.shape

(1, 1, 1, 1000)

In [18]:
np.argsort(pred.squeeze())[-3:]

array([282, 285, 287])

In [17]:
np.argsort(pred_proba.squeeze())[-3:]

array([282, 285, 287])

In [22]:
def create_label_lookup():
    with open('synset.txt', 'r') as f:
        label_list = [l.rstrip() for l in f]
    def _label_lookup(*label_locks):
        return [label_list[l] for l in label_locks]
    return _label_lookup

In [23]:
label_lookup = create_label_lookup()

In [38]:
labels=label_lookup(*np.flip(np.argsort(pred.squeeze()), 0)[:3])

In [43]:
dict(zip(labels, np.flip(np.sort(pred.squeeze()), 0)[:3]))

{'n02123159 tiger cat': 11.609917,
 'n02124075 Egyptian cat': 11.695245,
 'n02127052 lynx, catamount': 18.750937}